In [ ]:
import numpy as np
from scipy.fft import fft, fftfreq
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import FixedLocator, NullLocator, FixedFormatter

In [ ]:
config_IDs = 44,17
blob = {ID: np.load(f'../V2020_Rete_Sardegna_2021_06_03cr_stoch_tran_{ID}.npz', allow_pickle=True) for ID in config_IDs}
H = {ID: blob[ID]['inertia'].item() for ID in config_IDs}
E = {ID: blob[ID]['energy'].item() for ID in config_IDs}
M = {ID: blob[ID]['momentum'].item() for ID in config_IDs}

data = {ID: blob[ID]['data'].item() for ID in config_IDs}
time = {ID: blob[ID]['time'] for ID in config_IDs}
speed = {ID: data[ID]['gen']['s:xspeed'] for ID in config_IDs}
V = {ID: np.sqrt(data[ID]['bus']['m:ur']**2 + data[ID]['bus']['m:ui']**2) for ID in config_IDs}
Vnorm = {}
freq,Vf = {}, {}
for ID in config_IDs:
    n_samples = V[ID].shape[0]
    m = np.tile(V[ID].mean(axis=0), (n_samples, 1))
    s = np.tile(V[ID].std(axis=0), (n_samples, 1))
    Vnorm[ID] = (V[ID] - m) / s

In [ ]:
speedf, Vf = {}, {}
dur = 60 # [s]
dt = time[config_IDs[0]][1] - time[config_IDs[0]][0]
n_samples = int(dur/dt)
freq = fftfreq(n_samples, dt)[:n_samples//2]
for ID in config_IDs:
    speedf[ID] = np.zeros((freq.size, speed[ID].shape[1]))
    for j in range(speed[ID].shape[1]):
        n_trials = speed[ID].shape[0] // n_samples
        X = np.reshape(speed[ID][:n_trials*n_samples,j], (n_trials, n_samples))
        tmp = fft(X-1)
        speedf[ID][:,j] = np.mean(2.0 / n_samples * np.abs(tmp[:,:n_samples//2]), axis=0)
        
    Vf[ID] = np.zeros((freq.size, Vnorm[ID].shape[1]))
    for j in range(Vnorm[ID].shape[1]):
        n_trials = Vnorm[ID].shape[0] // n_samples
        X = np.reshape(Vnorm[ID][:n_trials*n_samples,j], (n_trials, n_samples))
        tmp = fft(X)
        Vf[ID][:,j] = np.mean(2.0 / n_samples * np.abs(tmp[:,:n_samples//2]), axis=0)

In [ ]:
fig,ax = plt.subplots(2, 1, sharex=True, figsize=(6,6))
j = 0
col = {44: 'k', 17: 'r'}
lw = 0.5
for ID in config_IDs:
    ax[0].plot(time[ID], speed[ID][:,j], col[ID], lw=lw, label='H = {:.2f}s'.format(H[ID]))
    ax[1].plot(time[ID], Vnorm[ID][:,j], col[ID], lw=lw)
ax[0].legend(loc='best')
ax[0].set_xlim([100, 120])
ax[-1].set_xlabel('Time [s]')
ax[0].set_ylabel('ω [p.u.]')
ax[1].set_ylabel('V norm.')
sns.despine()
fig.tight_layout()

In [ ]:
n_bins = 101
edges = np.linspace(-5, 5, n_bins)
N = {ID: np.zeros((n_bins-1, Vnorm[ID].shape[1])) for ID in config_IDs}
for ID in config_IDs:
    for j in range(Vnorm[ID].shape[1]):
        N[ID][:,j],_ = np.histogram(Vnorm[ID][:,j], bins=edges, density=True)

In [ ]:
rows,cols = 2,5
device_names = blob[17]['device_names'].item()['bus']
fig,ax = plt.subplots(rows, cols, figsize=(cols*2, rows*2), sharex=True, sharey=True)
for ID in config_IDs:
    names = blob[ID]['device_names'].item()['bus']
    for k,name in enumerate(device_names):
        i,j = k//cols, k%cols
        idx = names.index(name)
        ax[i,j].plot(edges[:-1], N[ID][:,idx], col[ID], lw=1)
        ax[i,j].set_title(name.split('___')[0], fontsize=8)
for a in ax[-1,:]:
    a.set_xlabel('V norm.')
for a in ax[:,0]:
    a.set_ylabel('PDF')
sns.despine()
fig.tight_layout()

In [ ]:
rows,cols = 2,5
device_names = blob[17]['device_names'].item()['bus']
fig,ax = plt.subplots(rows, cols, figsize=(cols*2, rows*2), sharex=True, sharey=True)
ticks = np.array([0.01, 0.1, 1, 10, 100])
for ID in config_IDs:
    names = blob[ID]['device_names'].item()['bus']
    for k,name in enumerate(device_names):
        i,j = k//cols, k%cols
        idx = names.index(name)
        ax[i,j].semilogx(freq, 20*np.log10(Vf[ID][:,idx]), col[ID], lw=0.5)
        ax[i,j].set_title(name.split('___')[0], fontsize=8)
        ax[i,j].xaxis.set_major_locator(FixedLocator(ticks))
        ax[i,j].xaxis.set_minor_locator(NullLocator())
        ax[i,j].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
        ax[i,j].set_xlim(ticks[[0,-1]])
sns.despine()
fig.tight_layout()

In [ ]:
rows,cols = 4,5
device_names = blob[17]['device_names'].item()['gen']
fig,ax = plt.subplots(rows, cols, figsize=(cols*2, rows*2), sharex=True, sharey=True)
ticks = np.array([0.01, 0.1, 1, 10, 100])
for ID in config_IDs:
    names = blob[ID]['device_names'].item()['gen']
    for k,name in enumerate(device_names):
        i,j = k//cols, k%cols
        idx = names.index(name)
        ax[i,j].semilogx(freq, 20*np.log10(1e-18 + speedf[ID][:,idx]), col[ID], lw=0.5)
        ax[i,j].set_title(name.split('___')[0], fontsize=8)
        ax[i,j].xaxis.set_major_locator(FixedLocator(ticks))
        ax[i,j].xaxis.set_minor_locator(NullLocator())
        ax[i,j].xaxis.set_major_formatter(FixedFormatter([f'{tick:g}' for tick in ticks]))
        ax[i,j].set_xlim(ticks[[0,-1]])
        ax[i,j].set_ylim([-200,-100])
sns.despine()
fig.tight_layout()